In [ ]:
import rasterio
import numpy as np
from rasterio.enums import Resampling
import os

upscale_factor = 10 # scale from 10m -> 1m
input_raster = "/media/remap/NO_HEAT_RB/City_Atlanta/Raw/LULC/2024_dw_composite_raw.tif"
resample_raster_path = "/media/remap/NO_HEAT_RB/City_Atlanta/Processed/resampled_lulc.tif"
output_raster = "/media/remap/NO_HEAT_RB/City_Atlanta/Processed/LULC/2024_LULC.tif"

os.makedirs(os.path.dirname(resample_raster_path), exist_ok=True)
os.makedirs(os.path.dirname(output_raster),exist_ok=True)

# # 1) Resample with nearest‐neighbor (for categorical LULC)
with rasterio.open(input_raster) as src:
    profile     = src.profile.copy()
    new_height  = src.height * upscale_factor
    new_width   = src.width  * upscale_factor
    new_transform = src.transform * src.transform.scale(
        (src.width  / new_width),
        (src.height / new_height)
    )
    profile.update({
        'height':    new_height,
        'width':     new_width,
        'transform': new_transform
    })
    data = src.read(
        out_shape=(src.count, new_height, new_width),
        resampling=Resampling.nearest
    )
    with rasterio.open(resample_raster_path, 'w', **profile) as dst:
        dst.write(data)

# 2) Reclassify on the *resampled* raster
with rasterio.open(resample_raster_path) as src:
    profile = src.profile.copy()
    profile.update(nodata=0)  # set LULC no‑data to 0
    with rasterio.open(output_raster, 'w', **profile) as dst: # no data as 0 in LULC
        for ji, window in src.block_windows(1):  # 1 is the band index
            data = src.read(1, window=window)
            original = data.copy()
            
            data = np.full_like(original, fill_value=0)
            
            data[np.isin(original, [1, 2, 3, 4, 5])] = 5  # grass
            data[original == 0] = 7                      # water
            data[original == 6] = 2                      # roof
            data[original == 7] = 6                      # baresoil
            data[original == 8] = 0                      # nodata          
            dst.write(data, window=window, indexes=1)